In [2]:
import psycopg2 as pg
import pandas as pd

connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb', password='adb2020')

<b>1. Znajdź listę wszystkich filmów o tej samej długości.</b>

In [35]:
df = pd.read_sql(f"SELECT length, title FROM film \
            GROUP BY length, title \
            ORDER BY length ", con=connection)
df

,length,title
0,46,Kwai Homeward
1,46,Alien Center
2,46,Iron Moon
3,46,Labyrinth League
4,46,Ridgemont Submarine
...,...,...
995,185,Gangs Pride
996,185,Worst Banger
997,185,Soldiers Evolution
998,185,Home Pity


Oto lista filmów zawierająca po kolei wszystkie filmy o tej samej długości.

<b>2. Znajdź wszystkich klientów mieszkających w tym samym mieście.</b>

In [39]:
df = pd.read_sql(f"SELECT city.city, COUNT(*) AS customer_sum FROM customer \
            LEFT JOIN address ON address.address_id = customer.address_id \
            LEFT JOIN city ON city.city_id = address.city_id \
            GROUP BY city.city \
            ORDER BY customer_sum DESC", con=connection)
df

,city,customer_sum
0,London,2
1,Aurora,2
2,Molodetno,1
3,Yerevan,1
4,Vitria de Santo Anto,1
...,...,...
592,Garden Grove,1
593,Novoterkassk,1
594,Asuncin,1
595,Maracabo,1


In [41]:
df = pd.read_sql(f"SELECT customer.first_name, customer.last_name, city.city FROM customer \
            LEFT JOIN address ON address.address_id = customer.address_id \
            LEFT JOIN city ON city.city_id = address.city_id \
            WHERE city.city IN('London','Aurora') \
            ORDER BY city.city", con=connection)
df

,first_name,last_name,city
0,Scott,Shelley,Aurora
1,Clinton,Buford,Aurora
2,Mattie,Hoffman,London
3,Cecil,Vines,London


Oto dwie pary klientów mieszkających w tym samym mieście.

<b>3. Oblicz średni koszt wypożyczenia wszystkich filmów.</b>

In [26]:
df = pd.read_sql(f"SELECT AVG(payment.amount) FROM film \
            LEFT JOIN inventory ON inventory.film_id = film.film_id \
            LEFT JOIN rental ON rental.inventory_id = inventory.inventory_id \
            LEFT JOIN payment ON payment.rental_id = rental.rental_id ", con=connection)
df

,avg
0,4.200606


<b>4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.</b>

In [27]:
df = pd.read_sql(f"SELECT category.name, COUNT(*) FROM film \
            LEFT JOIN film_category ON film_category.film_id = film.film_id \
            LEFT JOIN category ON category.category_id = film_category.category_id \
            GROUP BY category.name ", con=connection)
df

,name,count
0,Horror,56
1,Comedy,58
2,Sci-Fi,61
3,Drama,62
4,Foreign,73
5,Classics,57
6,Games,61
7,New,63
8,Travel,57
9,Music,51


<b>5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.</b>

In [34]:
df = pd.read_sql(f"SELECT country, COUNT(customer) AS customer_count FROM customer \
            LEFT JOIN address ON address.address_id = customer.address_id \
            LEFT JOIN city ON city.city_id = address.city_id \
            LEFT JOIN country ON country.country_id = city.country_id \
            GROUP BY country.country \
            ORDER BY customer_count DESC", con=connection)
df

,country,customer_count
0,India,60
1,China,53
2,United States,36
3,Japan,31
4,Mexico,30
...,...,...
103,Hungary,1
104,New Zealand,1
105,Senegal,1
106,Greenland,1


<b>6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.</b>

In [47]:
df = pd.read_sql(f"SELECT store.store_id, COUNT(customer.customer_id) FROM customer \
            LEFT JOIN store ON store.store_id = customer.store_id \
            GROUP BY store.store_id \
            HAVING COUNT(*) BETWEEN 100 AND 300", con=connection)
df

,store_id,count
0,2,273


<b>7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.</b>

In [5]:
df = pd.read_sql("SELECT customer.first_name, customer.last_name, SUM(film.length) FROM film \
                        LEFT JOIN inventory ON inventory.film_id = film.film_id \
                        LEFT JOIN rental ON rental.inventory_id = inventory.inventory_id \
                        LEFT JOIN customer ON customer.customer_id = rental.customer_id \
                        GROUP BY customer.customer_id \
                        HAVING SUM(film.length) > 200*60 ",con=connection)
df

,first_name,last_name,sum


Brak takich klientów. Rekordzista ma lekko ponad 58 godzin.

<b>8. Oblicz średnią wartość wypożyczenia filmu</b>

In [6]:
df = pd.read_sql("SELECT AVG(payment.amount) FROM payment", con=connection)
df

,avg
0,4.200606


<b>9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.</b>

In [11]:
df = pd.read_sql("SELECT category.name, AVG(film.length) FROM film \
                        LEFT JOIN film_category ON film_category.film_id = film.film_id \
                        LEFT JOIN category ON category.category_id = film_category.category_id \
                        GROUP BY category.name ",con=connection)
df

,name,avg
0,Horror,112.482143
1,Comedy,115.827586
2,Sci-Fi,108.196721
3,Drama,120.838710
4,Foreign,121.698630
5,Classics,111.666667
6,Games,127.836066
7,New,111.126984
8,Travel,113.315789
9,Music,113.647059


<b>10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.</b>

In [82]:
df = pd.read_sql("SELECT category.name, MAX(film.length) AS max_length FROM film \
                        LEFT JOIN film_category ON film_category.film_id = film.film_id \
                        LEFT JOIN category ON category.category_id = film_category.category_id \
                        GROUP BY category.name ",con=connection)
df

,name,max_length
0,Horror,181
1,Comedy,185
2,Sci-Fi,185
3,Drama,181
4,Foreign,184
5,Classics,184
6,Games,185
7,New,183
8,Travel,185
9,Music,185


<b>11. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.</b>

In [89]:
df = pd.read_sql("SELECT MAX(film.length) AS max_length FROM film",con=connection)
df

,max_length
0,185


In [91]:
df = pd.read_sql("SELECT film.title, film.length, category.name AS category FROM film \
                 LEFT JOIN film_category ON film_category.film_id = film.film_id \
                 LEFT JOIN category ON category.category_id = film_category.category_id \
                 WHERE film.length = 185",con=connection)
df

,title,length,category
0,Chicago North,185,Games
1,Control Anthem,185,Comedy
2,Darn Forrester,185,Action
3,Gangs Pride,185,Animation
4,Home Pity,185,Music
5,Muscle Bright,185,Travel
6,Pond Seattle,185,Animation
7,Soldiers Evolution,185,Sci-Fi
8,Sweet Brotherhood,185,Travel
9,Worst Banger,185,Action


Oto lista wszytkich najdłuższych filmów.

## Zadanie implementacyjne
Zaimplementuj wszystkie funkcje w pliku main.py zgodnie z opisem a następnie przetestuj je w notatniku.

In [2]:
import main

Funcje zostały zaimplementowane w pliku main.py i przetestowane przeze mnie przy użyciu podanych testów oraz automatycznie podczas commita do git classroom. Wszystkie testy przeszły pozytywnie.